# Apprentice chef 
 <b>Assignment 1</b> <br>
<i> Diego Gimenez </i>

<br><br>

<b>Introduction</b>

The Following analysis is focused on analyzing which factors are most important to successfully cross sell Half Way there to customers. This analysis includes a model to classify if a customer will successfully cross sell. 
<br> <br>

<b> Classification Model Notes: </b>
    <br><br>
False negatives (Recall) is the main classification metric for this model's business value. The opportunity cost of not offering Halfway There to someone that is willing to pay is higher than offering it to someone that would not purchase it. In addition, AUC was used for Grid Search model quality metric.
<br> <br>

<b> Target Variable Notes: </b> <br><br>

Halfway There, a cross-selling promotion where subscribers receive a half bottle of wine from a local California
vineyard every Wednesday (halfway through the work week).

<br> 
<b> Technical Notes </b>
- Seed across the notebook: 222 

## Import libraries and Read Data

In [1]:
import pandas as pd # Data Munging library
import matplotlib.pyplot as plt # Basic graphs
import seaborn as sns # Enhanced Graphs
import re # Regex
import numpy as np # Linear Algebra 

# Machine Learning Modules
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import recall_score, confusion_matrix,accuracy_score, auc, roc_auc_score

# Read CSVs
df = pd.read_excel('Apprentice_Chef_Dataset.xlsx')
dataDic = pd.read_excel('Apprentice_Chef_Data_Dictionary.xlsx')

## Helper Functions

<br>

The following helper functions have been developed and open-sourced due to the iterative nature of the Data Science Workflow. These helper functions are based on Chase Kusterer's Machine Learning course taught at Hult International Business School  

### Missing Value Flagger

In [2]:
def missingValueFlagger(dataframe):
    """ 
    Creates binary values of columns with missing values. 
    Such values are included in new column(s) with the format m_<column_name>.
    These columns will form part of the dataframe.
    -------------
    Params
        dataframe: dataframe to be used for the creation of flags. 
    """
    
    for col in dataframe:

        # creating columns with 1s if missing and 0 if not
        if dataframe[col].isnull().astype(int).sum() > 0:
            dataframe['m_'+col] = dataframe[col].isnull().astype(int)  

### outlierVisualEDA 

In [3]:
def outlierVisualEDA(dataframe,subfolderName):    
    '''
    This function creates distplots for all numerical variables. 
    The purpose of this helper function is to aid in visual EDA for outlier detection. 
    The analyst must choose by his/her intuition a cutoff point where the distribution looks normal. 
    ______________________________________________
    Params
        dataframe: Dataframe to plot numerical values
        subfolderName: Name of a folder in the current working directory to save the histograms. 
            THE SUBFOLDER MUST EXIST BEFORE RUNNING THIS HELPER FUNCTION
    
    
    '''
    
    plt.rcParams['figure.figsize']=(9,7)
    sns.set()
    
    lst_ = dataframe.select_dtypes(exclude=['object']).columns

    
    for i in range(len(lst_)):
        
        sns.distplot(dataframe[lst_[i]],
                 bins  = 'fd',
                 color = 'g')
        plt.xlabel(lst_[i])
        plt.savefig(f'{subfolderName}/{lst_[i]}_Histogram.png')
        plt.show()



### outlierFlagger

In [4]:

def outlierFlagger(dataframe, outlierDictionary):
    '''
This function creates outlier flags based on Visual Outler EDA. Please use the function outlierVisualEDA to do identify outliers.  
A flag consists of a binary vector column where 1 identifies an outlier and 0 does not identify an outlier. 
This function creates the outlier columns in the original dataframe. The column name format is the following: "Out_<ORIGINALCOLUMNNAME>" 
USER NEEDS TO CREATE THEIR OWN OUTLIER DICTIONARY.

This function is based on Chase Kusterer's Machine Learning course at Hult International Business School. 
---------------------------
Params: 
    dataframe: Dataframe to create flags on. 
    outlierDictionary: Dictionary containing column names as keys and numerical flag (outlier threshold) as values. The outlier dictionary keys
    most have the following format <ORIGINAL_COLUMN_NAME>_HI or <ORIGINAL_COLUMN_NAME>_LO. 
____________________________
Examples: 
# Dataframe 
exampleDf = pd.DataFrame({
                    'column1':[1,2,1,2,15],
                    'column2':[1,20,50,70,60] 
                    })
# outlierDictionary
outlierDic = {
                'column1_HI': 10,
                'column2_LO':2
            }

# Columns before outlier flagger
print(exampleDf.columns.to_list())

# Function call
outlierFlagger(exampleDf,outlierDic)

# Columns after outlier Flagger
print(exampleDf.columns.to_list())

# DataFrame with new columns
exampleDf

'''

    for key, value in outlierDictionary.items():

        # If it's a high outlier run the following
        if '_HI' in key:
            m = re.search(
                '\w+(?=_)', key
            )  # Extracts the original value of the column to be changed. If 'column1_HI', m will be 'column1'
            dataframe['Out_' + m.group()] = 0  # Creates flag column
            condition_hi = dataframe.loc[0:, 'Out_' + m.group()][
                dataframe[m.group()] >
                value]  # Subsets the rows that are greater than the
            # value provided in the trend change dictionary

            dataframe['Out_' + m.group()].replace(
                to_replace=condition_hi, value=1, inplace=True
            )  # Replaces the indices of condition_hi with 1 and puts it in the dataframe

        # If it's a low outlier run the following
        elif '_LO' in key:
            m = re.search(
                '\w+(?=_)', key
            )  # Extracts the original value of the column to be changed. If 'column1_LO', m will be 'column1'
            dataframe['Out_' + m.group()] = 0  # Creates flag column
            condition_lo = dataframe.loc[0:, 'Out_' + m.group()][dataframe[
                m.group()] < value]  # Subsets the rows that are lower than the
            # value provided in the trend change dictionary

            dataframe['Out_' + m.group()].replace(
                to_replace=condition_lo, value=1, inplace=True
            )  # Replaces the indices of condition_hi with 1 and puts it in the dataframe
        else:
            print(
                f'The following key from your outlierDictionary has not been properly named:{key} \n Remember that the format is <COLUMNNAME>_HI or <COLUMNNAME>_LO'
            )
            print("Call Gandalf, there's an error")

# # NOT RUN
# # Testing if the flags worked
# dfTest = df.copy()
# outlierFlagger(dfTest, outDic)
# dfTest[dfTest.loc[:,'TOTAL_PHOTOS_VIEWED_LO'] > 0][['TOTAL_PHOTOS_VIEWED','TOTAL_PHOTOS_VIEWED_LO']]


### trendBasedVisualEDA

In [5]:
def trendBasedVisualEDA(dataframe,targetVariableStr,subfolderName):
    '''
    This function creates scatter for all numerical variables with the target variable. 
    The purpose of this helper function is to aid in visual EDA for bivariate outlier detection. 
    The analyst must choose by his/her intuition a cutoff point where the distribution starts to deviate. 
    ______________________________________________
    Params
        dataframe: Dataframe to plot numerical values
        subfolderName: Name of a folder in the current working directory to save the histograms. 
            THE SUBFOLDER MUST EXIST BEFORE RUNNING THIS HELPER FUNCTION
    
    
    '''
    
    plt.rcParams['figure.figsize']=(9,7)
    sns.set()

    lst_ = dataframe.select_dtypes(exclude=['object']).columns

    
    for i in range(len(lst_)):
        sns.scatterplot(x = dataframe[lst_[i]],
                        y = dataframe[targetVariableStr],
                        color = 'b')
        plt.xlabel(lst_[i])
        plt.savefig(f'{subfolderName}/{lst_[i]}_Histogram.png')
        plt.show()

### trendBaseFlagger

In [6]:


def trendBaseFlagger(dataframe,trendChangeDictionary):
    import re
    
    """Summary line.
    
    This function creates trend-based flags based on Visual Trend-Based EDA. 
    Please use the function trendBasedVisualEDA to identify Trend-Based outliers that distorts the distribution of the variable.  
    A flag consists of a binary vector column where 1 identifies an outlier and 0 does not identify an outlier. 
    This function creates the trend change columns in the original dataframe. The column name format is the following: "Change_<ORIGINALCOLUMNNAME>" 
    USER NEEDS TO CREATE THEIR OWN TRENDCHANGE DICTIONARY.

    This function is based on Chase Kusterer's Machine Learning course at Hult International Business School. 
    ---------------------------
    Params: 
        dataframe: Dataframe to create flags on. 
        outlierDictionary: Dictionary containing column names as keys and numerical flag (outlier threshold) as values. The outlier dictionary keys
        most have the following format <ORIGINAL_COLUMN_NAME>_HI or <ORIGINAL_COLUMN_NAME>_LO. 
    ____________________________
    Example: 
    # Dataframe 
    exampleDf = pd.DataFrame({
                        'column1':[1,2,1,2,15],
                        'column2':[1,20,50,70,60] 
                        })
    # outlierDictionary
    outlierDic = {
                    'column1_HI': 10,
                    'column2_LO':2
                }

    # Columns before outlier flagger
    print(exampleDf.columns.to_list())

    # Function call
    outlierFlagger(exampleDf,outlierDic)

    # Columns after outlier Flagger
    print(exampleDf.columns.to_list())

    # DataFrame with new columns
    exampleDf
    ."""
    
    for key,value in trendChangeDictionary.items():
        
        # If it's a high outlier run the following
        if '_HI' in key:
            m = re.search('\w+(?=_)',key) # Extracts the original value of the column to be changed. If 'column1_HI', m will be 'column1'
            dataframe['Change_'+ m.group()] = 0 # Creates flag column 
            condition_hi = dataframe.loc[0:,'Change_'+ m.group()][dataframe[m.group()] > value] # Subsets the rows that are greater than the 
                                                                                                    # value provided in the trend change dictionary

            dataframe['Change_'+m.group()].replace(to_replace = condition_hi, # Replaces the indices of condition_hi with 1 and puts it in the dataframe
                                    value      = 1,
                                    inplace    = True)
        
        
        # If it's a low outlier run the following
        elif '_LO' in key:   
            
            m = re.search('\w+(?=_)',key)  # Extracts the original value of the column to be changed. If 'column1_LO', m will be 'column1'      
            
            dataframe['Change_'+ m.group()] = 0 #  Creates flag column
            
            condition_lo = dataframe.loc[0:,'Change_'+ m.group()][dataframe[m.group()] < value] # Subsets the rows that are lower than the 
                                                                                                    # value provided in the trend change dictionary
                
            dataframe['Change_'+ m.group()].replace(to_replace = condition_lo, # Replaces the indices of condition_hi with 1 and puts it in the dataframe
                                    value      = 1,
                                    inplace    = True)
        else: 
             print(f'The following key from your trendChangeDictionary has not been properly named:{key} \n Remember that the format is <COLUMNNAME>_HI or <COLUMNNAME>_LO')
             print("Call GanOutdalf, there's an error") 

## Data Exploration 


<br> 
<b>General Notes:</b>
<br>

This sections is focused on exploring the data at a high level. This includes: 
- Target's variable class imbalance. 
- Missing Value handling: Columns with NA's were flagged by creating a new column (m_Family_name). Original's column missing values were inputed by filling NAs as unknown. This is due that the only column with missing values is composed of nominal strings. 
- Data Measurement Classification: Features were divided in their measurement level (Continuous or Interval, Binary, Count, Categorical, Discrete or Nominal). I did this in order to segment the feature selection in section 4

<br>

_______________________

<br> 

<b>Hypothesis Section</b>
<br>

- Recall is better as a metric because we want avoid false negatives. 
- Accuracy is not the proper metric due to class imbalance. 


<br>

______________


Technical Notes:

- The dataset only contains null values in family name (2% of total data) 
- 3 floats, 22 integers, 4 objects
- FOLLOWED_RECOMMENDATIONS_PCT is a floats according to the data dictionary. In the dataset these columns are set as integers and all numbers are rounded to the nearest whole number. I assume that the data engineering team intended this. 




### Class Imbalance
Counts: 
- 1321 class 1
- 625 class 0

Almost the double of class 1 than class 0. This unbalance is not huge, but the training and test set will be stratified by the target variable in order to be misrepresent the data. 

In [7]:
# 1321 class 1
# 625 class 2

# df.CROSS_SELL_SUCCESS.value_counts()

In [8]:
# The following code has been commented out to not have information overload


# df.info() # To inspect datatypes
# FOLLOWED_RECOMMENDATIONS_PCT should be float 
# df.isnull().mean().round(2).sort_values(ascending = False) # Percentage of null values per column 
# df.head()

### Creation of missing values flags 
<br> 

The only missing values in the dataset are in the FAMILY_NAME column. Thus, it's passed to the missing value flagger helper function. 

<br> 

<b> Note:</b> The missingValueFlagger helper function creates populates the column m_FAMILY_NAME. This is a binary column where if 1, there's a missing value in family name, if not 0 

In [9]:
# Missing Value Flagger uses in place in the function            
missingValueFlagger(df)

# Filling NA's
df.loc[:,'FAMILY_NAME'].fillna('Unknown',inplace = True)


# df.columns # Checking which columns where created.


### Data Measurement Classification

<br>

I have divided categorized the dataset by data types in order to do feature specialized feature engineering by data type. 

<br> 
<br> 

<b>Heuristics of classification:</b> 
- Continuous or interval: Numerical values
- Binary: Contains 1 and 0
- Count: How many of something was measured.
- Categorical: I assume multinomial independent categories.
- Discrete / Nominal: Variables that are unique. 



<br>

#### Hypotheses 
<br>

- User that input mobile number might be more engaged? 
    - <b>THIS HYPOTHESIS WASN'T TRUE </b>

- Users that input taste and preferences are more engaged?
    - <b>THIS HYPOTHESIS WASN'T TRUE </b>
    
<br><br>
________
<br>
<b>Data Measurement classification:</b>
<br><br>

- Continuous or Interval
    - REVENUE
    - AVG_TIME_PER_SITE_VISIT 
    - FOLLOWED_RECOMMENDATIONS_PCT
    - AVG_PREP_VID_TIME
    - AVG_CLICKS_PER_VISIT


- Binary
    - MOBILE_NUMBER
    - TASTES_AND_PREFERENCES
    - PACKAGE_LOCKER
    - REFRIGERATED_LOCKER


- Count
    - TOTAL_MEALS_ORDERED
    - UNIQUE_MEALS_PURCH
    - CONTACTS_W_CUSTOMER_SERVICE
    - PRODUCT_CATEGORIES_VIEWED
    - CANCELLATIONS_BEFORE_NOON
    - CANCELLATIONS_AFTER_NOON
    - PC_LOGINS
    - MOBILE_LOGINS
    - WEEKLY_PLAN
    - EARLY_DELIVERIES
    - LATE_DELIVERIES
    - LARGEST_ORDER_SIZE
    - MASTER_CLASSES_ATTENDED
    - TOTAL_PHOTOS_VIEWED
    



- Categorical 
    - MEDIAN_MEAL_RATING



- Discrete or Nominal
    - Name 
    - Email
    - First Name
    - Last Name



## Feature Engineering

<br> 
<b>General Notes:</b>
<br>

This sections is focused on performing feature engineering. You will find the following subsections: 
- Domain Knowledge Feature engineering
    - **Meal Kit Industry Secondary Research**: Domain knowledge acquired by researching on internet and buying a meal kit in order to be in customer's shoes. 
    - **Cross Sell Sales Technique Secondary Research**: Domain Knowledge acquired by researching on internet. 
- **Outlier Feature Engineering**: Univariate and Bivariate Outlier analysis in order to identify anomalies or trends. Such anomalies were flagged by creating a binary column for each flagged feature. 
- **Discrete / Nominal Variable Feature Engineering**: Email types were featured by one hot encoding. 
- **Count Variables Feature Engineering**: All count variables were used to generate new features. 

<br>

_______________________

<br> 

<b>Hypothesis Section</b>
<br>

- I thought that count-variable and Univariate Outlier Feature Engineering was going to be successful because I captured anomalies in the data. This proved wrong.
- The most successful features were from Discrete/Nominal and Domain Knowledge sections. I believe that this is because cross sales depend a lot in sub-groups in the data. Capturing anomalies wouldn't do the trick because these were two sparse.
- One Bivariate Outlier feature proved to be useful in the final model. This might indicate that behavior of users that cross sell cannot be modeled by creating univariate features. For further analysis would be necessary to create more complex features (bivariate or multivariate) in order to test this hypothesis. 
- Many users have less total_meal_orders than Weekly plans. This is weird because for each weekly plan, the user receives at least 3 meals per week. I preferred to not create assumptions and to schedule a meeting with the analytics team to drill down more on this subject.  


<br>

______________


Technical Notes:

- Domain Knowledge Features were created by the following steps:
    1.  Simulate behavior stated by stakeholder (business case) using the available features. 
    2.  Calculate the mean of the target variable to serve as a benchmark (this was interpreted as the proportion of ones over zeros)
    3. Change the logic operators of these behaviors. 
    4. Calculate mean of CROSS_SELL_SUCCESS
    5. If mean is greater than benchmark mean (Step 2), flag is created and marked as a high (COLUMN_NAME_hi).
    6. If mean is less than benchmark mean (Step 2), flag is created and marked as a low (COLUMN_NAME_hi).
    
If mean was higher than benchmark mean I assumed that these users had a higher tendency to cross sell. The same logic is applied vice-versa. 
    



### Domain Knowledge Feature Engineering 

#### Meal Kit Industry Secondary Research
<br>

Many of the count variables feature engineering has been based on heuristics extracted from secondary research. 
<br>


- <b>What Does the Customer Really Value in a Meal Kit?</b>
    - Nearly 6 out of 10 consumers reported they quit their meal kit subscription service because the value wasn’t worth the money.
    - Moreover, consumers are more likely to stick with a meal kit if it aligns with their dietary goals.
- <b>Meal Kit Companies scramble to dodge disaster</b>
    - Gist of the article
        - Meal kit companies face a logistics nightmare. Because it's perishable and varies a lot. 
        - “Fresh food by its nature is perishable inventory,” said Rett Wallace of Triton Research, a provider of data on technology companies. “It’s easy to see           how marketing and delivery costs would zero out any margin they make on the food itself.”
- <b>Packaged Facts article </b> <br>
    <i>Verbatim</i>: The same poll reported 46% of respondents saying time management was a deciding factor in purchasing meal kit delivery. 44% of respondents said they would reconsider purchasing meal kits purely because of convenience.
    - Time delivery is very important! 
        - Hypothesis: Create features based on how late meal kit is arriving - LATE_DELIVERIES. This might capture variations in revenue. 
- <b> Hello Fresh Grabs bigger slice of shrinking meal <b> <br>
    - <i> Verbatim: </i>  “There is less prep time with it. All you do is chop, and everything is prepackaged and well-labeled. Directions are super simple,” Mr. Yarbro said. He has discontinued Blue Apron and plans to stick to HelloFresh.
        - Preparation time and how easy is to do it is very important. 
        - Hypothesis:
            - Create feature using AVG_PREP_VID_TIME. Avg prep time can capture how complicated is to do a meal
- <b> Blue Shield Sheds customers as meal kit competition mounts <b>
    - <i> Verbatim: </i> Mr. Dickerson said customer retention has improved as Blue Apron has introduced meals that are easier to make, added more variety and beef to its offerings, and struck partnerships to offer recipes that match dining trends, such as the Whole30 elimination diet.
    - This corroborates previous point. Easiness to cook is important. How we can measure this by the count variables that we have? 
        
        
    
_________
<br> 
- Sources
    - https://www.ehy.com/food-agency/insights/what-does-the-customer-really-value-in-a-meal-kit/ 
    - https://www.wsj.com/articles/meal-kit-companies-scramble-to-dodge-disaster-1532700025
    - https://www.packagedfacts.com/Content/Featured-Markets/Meal-Kit-Delivery-Services
    - https://www.wsj.com/articles/hellofresh-grabs-bigger-slice-of-shrinking-meal-kit-market-11577279059
    - https://www.wsj.com/articles/blue-apron-sheds-customers-as-meal-kit-competition-mounts-1518528989

In [36]:
# df.WEEKLY_PLAN.describe() # Count of weeks that a customer subscribed 

# It seems that users are not using the weekly meal plans! 
# df[df.WEEKLY_PLAN == 52]
# df.TOTAL_MEALS_ORDERED.describe()

# It looks that there's no effective cross sell success for people that order a lot of meals
# df.loc[df.TOTAL_MEALS_ORDERED > 95, ['WEEKLY_PLAN',
#                                      'NAME',
#                                      'TOTAL_MEALS_ORDERED',
#                                     'CROSS_SELL_SUCCESS']].sort_values('WEEKLY_PLAN',ascending = False).groupby('CROSS_SELL_SUCCESS').mean()

# It looks that the Total Meals people that has ordered less than 60 meals are slightly more prone to do succesfull cross sell  
# df.loc[df.TOTAL_MEALS_ORDERED < 60, ['WEEKLY_PLAN',
#                                      'NAME',
#                                      'TOTAL_MEALS_ORDERED',
#                                     'CROSS_SELL_SUCCESS']].sort_values('WEEKLY_PLAN',ascending = False).groupby('CROSS_SELL_SUCCESS').mean()

#### Cross Sell Sales Technique Domain knowledge research

<b> Hubspot: </b>
- Definition of Cross Selling
    - Cross-selling is encouraging the purchase of anything in conjunction with the primary product. For example, if a customer has already purchased a subscription to your marketing tool, cross-selling would encourage that customer to purchase a subscription to your CRM.
        - Hypothesis: Engagement can be leveraged to improve cross sell? What encourages customers to cross sell?
- How to effectively cross sell according to hubspot
    -  <u>Get to know your audience</u> : create personas for your customers and understand their goals and their challenges in order to identify the products you could cross-sell and upsell that are most useful to them.
    - <u> Think about problems and offer solutions that map to the products </u>: Review your product offerings and try to align them with your customer journey. That way, you'll have a clear idea of common challenges your customers face, and exactly which of your products you can try to cross-sell or upsell as a possible solution.
        - Hypotheses from section: Understanding customers' pain points and understanding their goal for the product usage are key to identify cross selling. 
        - Pain point Variables: 
            - Low Median Meal Ratings: Categorize this variable! 
            - High Late Deliveries: More late deliveries, more pain. Also, this is the biggest hurdle for meal kit companies
            - Refrigerated locker: Users without this type of locker might have a bad experience. 
        - Cross Sell and customer Journey: 
            - Recreate customer journey
                - Hypothesis: People that register with a landline number are older. Probably these people tend to buy less wine because they don't want to drink half a bottle of wine on a Wednesday . 
                
<br> 

<b> EConsultancy <b> 
- Focus cross selling in complementary products: There is no point offering people something they are simply not going to be interested in. This is a waste of everyone’s time and a missed opportunity for a sale. You should always offer the customer something that compliments the product they are already interested in. 
- Ways to successfully cross-sell in e-commerce: 
    - Keep it relevant: You should always offer the customer something that compliments the product they are already interested in. The easiest and most obvious way to do this is to offer a product that directly relates to the one the customer is already looking at or has added to their basket. 
        - Hypothesis: Probably users in different parts of their journey have a higher tendency on cross selling. It would be interesting creating a longevity feature 
    - Offer a deal: You can go one step further than simply presenting additional products and offer a discount if people buy them as a ‘bundle’.

_____________

<br>

 References:
 - https://blog.hubspot.com/service/cross-selling
 -  https://econsultancy.com/cross-selling-online-why-it-s-important-how-to-do-it/

##### Customer Journey

I'm going to recreate the customer journey with the available data in order to identify challenges that users might have. This is based on the hubspot article.  

###### New Users
<br>
New-user customer journey of Apprentice Chef:

1. Go to the website or download the mobile app.
2. Register using a phone number. Confirm the phone number with a one-time code sent
via SMS message or voice call.
3. (optional) Select tastes and preferences in terms of food and specify dietary
restrictions/food allergies.
4. (optional) May submit a picture of a government-issued ID card if also wanting to
purchase alcohol.
5. (optional) Register for a weekly meal plan and receive a discount on all purchases.
6. Select meals and set meal delivery schedule.
7. (optional) Fill in feedback survey on order.

<b> Hypotheses </b>

 - Users that didn't say their preferences and Didn't ave a weekly plan
     - These users convert 62% of times in avg 
 
 - Users that say their preferences and haven't registered to weekly plans 
     - Tend to convert 74% of times in avg
 
- Users that say their preferences and registered to 10 or more weekly plans 
    - Tend to convert 70% of times in avg

In [37]:
######## New users customer journey featuring ############

############ Go to the website or download mobile app #################
##df.MOBILE_LOGINS + df.PC_LOGINS
df['uses_mobile_app'] = 0
df.loc[df.MOBILE_LOGINS > 0, 'uses_mobile_app'] = 1

######## SELECT TASTS AND PREFERENCES ################
##df.TASTES_AND_PREFERENCES

########## REGISTER FOR WEEKLY PLAN #####################

## Users that didn't say their preferences and Didn't ave a weekly plan
## These users convert 62% of times in avg 
# df[(df.MOBILE_NUMBER == 1) & df[(df.MOBILE_LOGINS == 1) | (df.TASTES_AND_PREFERENCES == 0) | (df.WEEKLY_PLAN == 0) ].describe().round(2)

## Users that say their preferences and haven't registered to weekly plans 
## Tend to convert 74% of times in avg
# df[((df.MOBILE_NUMBER == 1) & df.TASTES_AND_PREFERENCES == 1) & (df.WEEKLY_PLAN == 0) ].describe().round(2)

## Users that say their preferences and registered to 10 or more weekly plans 
## Tend to convert 70% of times in avg
# df[ (df.TASTES_AND_PREFERENCES == 1) & (df.WEEKLY_PLAN > 10) ].describe().round(2)

########### More in depth Analysis of Weekly Plan and possible relationship with total meals ordered ###########3
    ## I can't see anything valuable by combining weekly plan and Total Meals ordered!! 
# df.loc[(df.TOTAL_MEALS_ORDERED / df.WEEKLY_PLAN) > 3,  [ 'WEEKLY_PLAN','TOTAL_MEALS_ORDERED']].sort_values('WEEKLY_PLAN')
# df.groupby('WEEKLY_PLAN')

######### Valuable Feature #############333
##  High New Engaged Customer
df['high_new_customer_engagement'] = 0

df.loc[(df.MOBILE_NUMBER == 1) & (df.TASTES_AND_PREFERENCES == 1) & (df.WEEKLY_PLAN == 0),'high_new_customer_engagement' ] = 1



###### Existing Users
<br>
Existing-user customer journey of Apprentice Chef:


1. Log into the website or mobile app.
2. (optional) Update profile and/or orders.
3. Select meals and confirm meal delivery schedule.
4. (optional) Fill in feedback survey on order.
5. Food is delivered, cooked, and rated (created by me)


Funnel was analyzed per section because the number of datapoints was highly reduced if the analysis was performed in a combined manner. 

In [13]:

######## 1. Log into the website or mobile app ########### 
## df.MOBILE_LOGINS.describe()
## df.PC_LOGINS.describe()
df[(df.MOBILE_LOGINS > 1)  & (df.PC_LOGINS > 1 )] # least engaged users from first part of funnel
# First part of funnel 65% of users convert 
df['existing_users_1_funnel_lo'] = 0
df.loc[(df.MOBILE_LOGINS > 1)  & (df.PC_LOGINS > 1 ),'existing_users_1_funnel_lo'] = 1
# df['existing_users_1_funnel_lo'].fillna(0, inplace = True)


######### 2. (optional) Update profile and/or orders. #############
df.AVG_CLICKS_PER_VISIT
df.PRODUCT_CATEGORIES_VIEWED
df[(df.AVG_CLICKS_PER_VISIT < 13) & (df.AVG_TIME_PER_SITE_VISIT > 94)]
df['existing_users_2_funnel_hi'] = 0
df.loc[(df.AVG_CLICKS_PER_VISIT < 13) & (df.AVG_TIME_PER_SITE_VISIT > 94),'existing_users_2_funnel_hi'] =1
# df['existing_users_2_funnel_hi'].fillna(0, inplace = True)

########3 3. Select meals and confirm meal delivery schedule. ########33
# df.TOTAL_MEALS_ORDERED
# df.AVG_TIME_PER_SITE_VISIT
# df[(df.TOTAL_MEALS_ORDERED > 40) & (df.AVG_TIME_PER_SITE_VISIT > 94)]
df['existing_users_3_funnel_hi'] = 0
df.loc[(df.TOTAL_MEALS_ORDERED > 40) & (df.AVG_TIME_PER_SITE_VISIT > 94),'existing_users_3_funnel_hi'] = 1
# df['existing_users_3_funnel_hi'].fillna(0, inplace = True)



# Final part of Funnel 
## df.LATE_DELIVERIES
## df.EARLY_DELIVERIES
## df.FOLLOWED_RECOMMENDATIONS_PCT
## df.MEDIAN_MEAL_RATING

# Mean of cross sell 0.46, can be useful to extract low variance
df['existing_users_final_funnel_lo'] = 0
df.loc[(df.LATE_DELIVERIES > 40) |\
       (df.EARLY_DELIVERIES > 94) |\
       (df.FOLLOWED_RECOMMENDATIONS_PCT < 40)\
       , 'existing_users_final_funnel_lo'] = 1

df['existing_users_final_funnel_hi'] = 0
df.loc[(df.LATE_DELIVERIES > 40) |\
       (df.EARLY_DELIVERIES > 94) |\
       (df.FOLLOWED_RECOMMENDATIONS_PCT > 40),\
       'existing_users_final_funnel_hi'] = 1

df['existing_users_final_funnel_hi_ALTERNATIVE'] = 0
df.loc[(df.LATE_DELIVERIES > 40) |\
       (df.EARLY_DELIVERIES > 94) |\
       (df.FOLLOWED_RECOMMENDATIONS_PCT < 30), \
       'existing_users_final_funnel_hi_ALTERNATIVE'] = 1

## df.loc[(df.TOTAL_MEALS_ORDERED < 90) & (df.AVG_TIME_PER_SITE_VISIT < 40),'CROSS_SELL_SUCCESS'].describe()

df['tot_meal_per_week'] = (df.TOTAL_MEALS_ORDERED / df.loc[df.WEEKLY_PLAN > 0,'WEEKLY_PLAN'])
df['tot_meal_per_week'].fillna(0, inplace = True)

#### Outlier Feature Engineering

##### Univariate Outlier Feature Engineering 
This features didn't work. Hypothesis is explained section 4.

In [14]:
# sns.catplot(x = 'CROSS_SELL_SUCCESS',
#                             y = 'REFRIGERATED_LOCKER',
#                             data = df,
#            kind = 'violin')
outlierFlagger(df,{
                  'MEDIAN_MEAL_RATING_LO': 3, # It looks that it doesn't descriminate well the variance
                  'LATE_DELIVERIES_HI': 1 # It looks that there is not much difference between users. 
                  
                  })


# sns.boxplot(x = df['CROSS_SELL_SUCCESS'],
#                             y = df['REFRIGERATED_LOCKER'],
#                             color = 'b')

# sns.catplot(x = 'CROSS_SELL_SUCCESS',
#                             y = 'REFRIGERATED_LOCKER',
#                             data = df,
#            kind = 'violin')

# Double of the users that don't have a fridge decide to buy more wine!!
# I don't know how to feature this out, but this can be a useful insight afterwards.
# I need to dig in more into this
# pd.pivot_table(df, values = 'CROSS_SELL_SUCCESS', index =  'REFRIGERATED_LOCKER', aggfunc= len ) # There are more values in the 0 


# # Here we can see that from the ones that don't have refrigerated locker, converted more.
# This also can be due to the fact that not that many people have refrigerated lockers
# df.loc[df.REFRIGERATED_LOCKER == 0 ,'CROSS_SELL_SUCCESS'].hist()
# df.groupby('REFRIGERATED_LOCKER').count()


###### Outlier Analysis

- TLDR; AVG_TIME_PER_SITE_VISIT, CANCELLATIONS_BEFORE_NOON, TOTAL_PHOTOS_VIEWED have a very high value on 100% quantile

<br>


- AVG_TIME_PER_SITE_VISIT: The 100% percentile is 15 times greater than the 0.99. This is weird
-  CANCELLATIONS_BEFORE_NOON: The 100% percentile is 2 times greater than the 0.99 percentile. This is not very weird but worth inspecting. 
-  TOTAL_PHOTOS_VIEWED: The 100% percentile is ~ 2 times greater than the 0.99 percentile. This is not very weird but worth inspecting. 

In [16]:
# We use the loc to look more in depth in a variable
# df.iloc[:, 14:].quantile([0.20,
#                          0.40,
#                          0.60,
#                          0.80,
#                          1.00])

# Extreme Variables Analysis based on quantiles

# lst_ = ['AVG_TIME_PER_SITE_VISIT','CANCELLATIONS_BEFORE_NOON','TOTAL_PHOTOS_VIEWED']

# for col in lst_:
#     print(df.loc[:, col].quantile([ 0.90,
#                                       0.95,
#                                       0.99,
#                                       1.00]),'\n___________________')
    
# # Output 
# 0.90     146.3950
# 0.95     179.6825
# 0.99     248.5150
# 1.00    1645.6000
# Name: AVG_TIME_PER_SITE_VISIT, dtype: float64 
# ___________________
# 0.90     3.0
# 0.95     4.0
# 0.99     7.0
# 1.00    13.0
# Name: CANCELLATIONS_BEFORE_NOON, dtype: float64 
# ___________________
# 0.90     336.00
# 0.95     471.50
# 0.99     769.65
# 1.00    1600.00
# Name: TOTAL_PHOTOS_VIEWED, dtype: float64 
# ___________________



# I want to check the variables that cought my attention on the quantile analysis. 
# AVG_TIME_PER_SITE_VISIT: The 100% percentile is 15 times greater than the 0.99. This is weird
# CANCELLATIONS_BEFORE_NOON: The 100% percentile is 2 times greater than the 0.99 percentile. This is not very weird but worth inspecting. 
# TOTAL_PHOTOS_VIEWED: The 100% percentile is ~ 2 times greater than the 0.99 percentile. This is not very weird but worth inspecting. 

# df[df['AVG_TIME_PER_SITE_VISIT'] > 1600].iloc[:,0:15]


###### Outlier Analysis - Visual EDA

- KDE plots where done for each column and outliers flags were created in new columns with the format Out_<COLUMN_NAME>

In [17]:
# outlierVisualEDA(df,'hists') # Commented out to avoid content overload

###### Outlier Flags

   The following outlier flags were chosen based on the following heuristics: 
 - Zero-inflated: If a variable have a spike on number of zeros, it was flagged. 
 - Normal Distribution approximation: Outliers that deform the distribution of the feature. Outliers were flagged to make the feature have a normal distribution.  

In [18]:
# Flags created after doing Visual Outlier Analysis EDA 
outDic = {
    'REVENUE_HI':800,
    'REVENUE_HI':2200,
    'AVG_PREP_VID_TIME_HI': 230,
    'AVG_PREP_VID_TIME_LO': 80,
    'AVG_TIME_PER_SITE_VISIT_HI': 300,
    'CANCELLATIONS_AFTER_NOON_HI': 1,
    'CANCELLATIONS_BEFORE_NOON_HI': 5,
    'CONTACTS_W_CUSTOMER_SERVICE_HI': 8,
    'EARLY_DELIVERIES_LO': 0,
    'EARLY_DELIVERIES_HI': 5,
    'LARGEST_ORDER_SIZE_HI': 7,
    'LATE_DELIVERIES_HI': 5,
    'MASTER_CLASSES_ATTENDED_HI': 1,
    'MEDIAN_MEAL_RATING_LO': 2,
    'MEDIAN_MEAL_RATING_HI': 4,
    'MOBILE_LOGINS_LO': 1,
    'MOBILE_LOGINS_HI': 2.5,
    'PC_LOGINS_LO': 5,
    'PC_LOGINS_HI': 6,
    'TOTAL_MEALS_ORDERED_HI': 200,
    'TOTAL_MEALS_ORDERED_LO': 25,
    'TOTAL_PHOTOS_VIEWED_HI': 500,
    'TOTAL_PHOTOS_VIEWED_LO': 1,
    'UNIQUE_MEALS_PURCH_HI': 9,
    'WEEKLY_PLAN_HI': 12,
    'WEEKLY_PLAN_LO': 1,
    'AVG_CLICKS_PER_VISIT_LO': 8
}


outlierFlagger(df, outDic)

##### Bivariate Feature Engineering

The following analysis has been done in order to explore bivariate relationships between the features and the target variable.
The purpose of this visual EDA is to detect outliers. I chose the cutoff points where the distribution starts to deviate.

###### Trend-Based Outlier Visual EDA

This Section Contains the Boxplots of all features grouped by each class of the target variable. The thresholds were chosen from this analysis.

In [19]:
# Commented out to not have content overload

# trendBasedVisualEDA(df,'REVENUE','trendBased')

# trendBasedVisualEDA(df, 'CROSS_SELL_SUCCESS','trendBased',binaryTarget=True)



###### Trend-Based Flag Creation 
<br>
Flags from visual trend-based visual EDA are  defined in the following dictionary

In [20]:

trendBaseDic = {
	'CANCELLATIONS_BEFORE_NOON_HI': 1,
    'FOLLOWED_RECOMMENDATIONS_PCT_LO': 20
    
    
}
            
trendBaseFlagger(df,trendBaseDic)

### Discrete / Nominal Variable Feature engineering 

In the following section I featured out the Family Name, First Name, and emails. 

These features were relevant in the final model. Explained in section 4.


In [21]:
# Users that have their family name equal to their first name have any impact on the target?
# dfSameNameFamilyName = df[df['FIRST_NAME'] == df['FAMILY_NAME']] They don't

# Divide the emails by domain knowledge provided by Customer Service Team
professional = [
    "mmm.com", "amex.com", "apple.com", "boeing.com", "caterpillar.com",
    "chevron.com", "cisco.com", "cocacola.com", "disney.com", "dupont.com",
    "exxon.com", "ge.org", "goldmansacs.com", "homedepot.com", "ibm.com",
    "intel.com", "jnj.com", "jpmorgan.com", "mcdonalds.com", "merck.com",
    "microsoft.com", "nike.com", "pfizer.com", "pg.com", "travelers.com",
    "unitedtech.com", "unitedhealth.com", "verizon.com", "visa.com",
    "walmart.com"
]

personal = ["gmail.com", "yahoo.com", "protonmail.com"]

junk = [
    "me.com", "aol.com", "hotmail.com", "live.com", "msn.com", "passport.com"
]

# Split by @ and select domains
df['EMAIL_DOMAIN'] = df.EMAIL.str.split('@', expand=True)[1]

# Create email_tue column
df['EMAIL_TYPE'] = '0'

# Populate the Email Type column by if else statements
for index, row in df.iterrows():

    if df.loc[index, 'EMAIL_DOMAIN'] in professional:
        df.loc[index, 'EMAIL_TYPE'] = 'professional'
    elif df.loc[index, 'EMAIL_DOMAIN'] in personal:
        df.loc[index, 'EMAIL_TYPE'] = 'personal'
    elif df.loc[index, 'EMAIL_DOMAIN'] in junk:
        df.loc[index, 'EMAIL_TYPE'] = 'junk'
    elif df.loc[index, 'EMAIL_DOMAIN'] not in professional or df.loc[
            row, 'EMAIL'] not in personal or df.loc[row, 'EMAIL'] not in junk:
        df.loc[index, 'EMAIL_TYPE'] = 'unknown'
    else:
        print('Call Gandalf, there has been a problem')

email_dummies = pd.get_dummies(df.EMAIL_TYPE, drop_first=True)

df = pd.concat([df, email_dummies], axis=1)



### Count Variable Feature Engineering

The following section contains Count-Variable feature engineering. Many of these variables are important in the final model. 

These features were created based on domain knowledge from the section 4.1 and by trial and error. 

<br><br>
<b> Count Variables </b> 
_____________

- CANCELLATIONS_AFTER_NOON,
- CANCELLATIONS_BEFORE_NOON,
- CONTACTS_W_CUSTOMER_SERVICE,
- EARLY_DELIVERIES,
- LARGEST_ORDER_SIZE,
- LATE_DELIVERIES,
- MASTER_CLASSES_ATTENDED,
- MOBILE_LOGINS,
- PC_LOGINS,
- PRODUCT_CATEGORIES_VIEWED,
- TOTAL_MEALS_ORDERED,
- TOTAL_PHOTOS_VIEWED,
- UNIQUE_MEALS_PURCH,
- WEEKLY_PLAN

In [22]:
df['late_over_total'] = df.LATE_DELIVERIES / df.TOTAL_MEALS_ORDERED # Late Deliveries per how many meals the user ordered in the year
df['early_over_total'] = df.EARLY_DELIVERIES / df.TOTAL_MEALS_ORDERED #  Late Deliveries per how many meals the user ordered in the year
df['product_cat_viewed_over_total_logins'] = df.PRODUCT_CATEGORIES_VIEWED / (df.PC_LOGINS + df.MOBILE_LOGINS) # Categories viewed per total number of logins 
df['late_deliveries_over_product_catViewed'] = df.LATE_DELIVERIES / df.PRODUCT_CATEGORIES_VIEWED # Late Deliveries per categories viewed
df['total_logins_over_total_meals_ordered'] = (df.MOBILE_LOGINS + df.PC_LOGINS) / df.TOTAL_MEALS_ORDERED # Total Logins per total meals ordered


df['total_unique_orders'] =  df.UNIQUE_MEALS_PURCH / df.TOTAL_MEALS_ORDERED # Total Unique orders
df['total_orders_contact_custServ'] = df.CONTACTS_W_CUSTOMER_SERVICE /  df.TOTAL_MEALS_ORDERED # Total orders that contacted the customer service
df['total_orders_without_cancellations'] = df.TOTAL_MEALS_ORDERED - df.CANCELLATIONS_BEFORE_NOON #Total orders that haven't had any cancellation
df['late_deli_over_total_meals'] = df.LATE_DELIVERIES / df.TOTAL_MEALS_ORDERED # Late Deliveries over total meals ordered
df['total_logins'] = df.PC_LOGINS + df.MOBILE_LOGINS # Total Logins

# This Feature  is categorical 
# Median Meal Dummy Variable


medianMealDummy = pd.get_dummies(df['MEDIAN_MEAL_RATING'],
                                  drop_first=True)

medianMealDummy.columns = ['median_meal_2','median_meal_3','median_meal_4','median_meal_5']

df = df.drop('MEDIAN_MEAL_RATING', axis = 1)

df = df.join(medianMealDummy)



## Modelling 

<br> 

The following section contains: 
- Base Model: Logistic Regression
- Model Tournament: Lasso, Ridge, LinearRegression, ElasticNet, ARDRegression, AdaBoostRegressor, BaggingRegressor, GradientBoostingRegressor,RandomForestRegressor
- Winner: Logistic Regression with Hyper Parameter Tuning

<br>

_______

<br>

<b>Results:<b>

~~~
Train Metrics:
            Recall:     0.851 
            Accuracy:   0.777
            AUC:        0.735 
            
____________________________________________________________________
            
    Test Metrics:
            Recall:     0.819 
            Accuracy:   0.749
            AUC:        0.711 
~~~

<br>

_______

<br>

<b>Notes:</b>
- I used different preprocessing techniques (StandardScaler, MixManScaler) and Dimensionality Reduction (PCA). None of these proved to improve the final model. 

### Feature sets
<br> 

The Following feature sets were developed by different feature selection approaches:
- Ensemble First Prune: Pruned full dataset by selecting features that had a relative importance higher than 0 (Gradient Boosting)
- Logistic First Prube: Pruned full dataset by selecting features that had a coefficient lower than Zero (Logistic Regression)
- First_log_reg_select_from_model_sklearn: Pruned full dataset by selecting features obtained by SelectFromModel method from SkLearn
- Second_log_reg_select_from_model_sklearn: Pruned full First_log_reg_select_from_model_sklearn Features by selecting features obtained by SelectFromModel method from SkLearn
- first_dec_tree_prune: Pruned full dataset by selecting features that had a relative importance higher than 0 (Decision Tree)

In [23]:
features = {

    # Worse than full model on Logistic Regression
'ensemble_first_prune': ['REVENUE',
 'TOTAL_MEALS_ORDERED',
 'UNIQUE_MEALS_PURCH',
 'CONTACTS_W_CUSTOMER_SERVICE',
 'AVG_TIME_PER_SITE_VISIT',
 'MOBILE_NUMBER',
 'CANCELLATIONS_BEFORE_NOON',
 'CANCELLATIONS_AFTER_NOON',
 'TASTES_AND_PREFERENCES',
 'PC_LOGINS',
 'MOBILE_LOGINS',
 'WEEKLY_PLAN',
 'LATE_DELIVERIES',
 'PACKAGE_LOCKER',
 'REFRIGERATED_LOCKER',
 'FOLLOWED_RECOMMENDATIONS_PCT',
 'AVG_PREP_VID_TIME',
 'LARGEST_ORDER_SIZE',
 'MASTER_CLASSES_ATTENDED',
 'AVG_CLICKS_PER_VISIT',
 'TOTAL_PHOTOS_VIEWED',
 'high_new_customer_engagement',
 'personal',
 'professional',
 'Out_CONTACTS_W_CUSTOMER_SERVICE',
 'Out_WEEKLY_PLAN',
 'late_over_total',
 'early_over_total',
 'product_cat_viewed_over_total_logins',
 'late_deliveries_over_product_catViewed',
 'total_logins_over_total_meals_ordered',
 'total_unique_orders',
 'total_orders_contact_custServ',
 'total_orders_without_cancellations',
 'late_deli_over_total_meals',
 'median_meal_3',
 'median_meal_4'],
    
    # Worse than full model on logistic regression
'logistic_first_prune': ['REVENUE',
 'TOTAL_MEALS_ORDERED',
 'UNIQUE_MEALS_PURCH',
 'CONTACTS_W_CUSTOMER_SERVICE',
 'PRODUCT_CATEGORIES_VIEWED',
 'AVG_TIME_PER_SITE_VISIT',
 'CANCELLATIONS_AFTER_NOON',
 'PC_LOGINS',
 'WEEKLY_PLAN',
 'EARLY_DELIVERIES',
 'LATE_DELIVERIES',
 'PACKAGE_LOCKER',
 'REFRIGERATED_LOCKER',
 'FOLLOWED_RECOMMENDATIONS_PCT',
 'AVG_PREP_VID_TIME',
 'LARGEST_ORDER_SIZE',
 'MASTER_CLASSES_ATTENDED',
 'AVG_CLICKS_PER_VISIT',
 'TOTAL_PHOTOS_VIEWED',
 'Out_AVG_PREP_VID_TIME',
 'Out_AVG_TIME_PER_SITE_VISIT',
 'Out_CANCELLATIONS_AFTER_NOON',
 'Out_CANCELLATIONS_BEFORE_NOON',
 'Out_CONTACTS_W_CUSTOMER_SERVICE',
 'Out_EARLY_DELIVERIES',
 'Out_LARGEST_ORDER_SIZE',
 'Out_LATE_DELIVERIES',
 'Out_MASTER_CLASSES_ATTENDED',
 'Out_MEDIAN_MEAL_RATING',
 'Out_MOBILE_LOGINS',
 'Out_PC_LOGINS',
 'Out_TOTAL_MEALS_ORDERED',
 'Out_TOTAL_PHOTOS_VIEWED',
 'Out_UNIQUE_MEALS_PURCH',
 'Out_WEEKLY_PLAN',
 'Out_AVG_CLICKS_PER_VISIT']
    
    # AUC:  Train  0.774 , Test 0.735 
,'First_log_reg_select_from_model_sklearn':['MOBILE_NUMBER',
 'CANCELLATIONS_AFTER_NOON',
 'TASTES_AND_PREFERENCES',
 'REFRIGERATED_LOCKER',
 'MASTER_CLASSES_ATTENDED',
 'high_new_customer_engagement',
 'existing_users_3_funnel_hi',
 'existing_users_final_funnel_lo',
 'existing_users_final_funnel_hi',
 'existing_users_final_funnel_hi_ALTERNATIVE',
 'premium_weekly_plan',
 'Out_TOTAL_PHOTOS_VIEWED',
 'Change_CANCELLATIONS_BEFORE_NOON',
 'Change_FOLLOWED_RECOMMENDATIONS_PCT',
 'personal',
 'professional',
 'median_meal_3']
    
    #
   ,'Second_log_reg_select_from_model_sklearn': ['MOBILE_NUMBER',
 'existing_users_final_funnel_lo',
#  'existing_users_final_funnel_hi',
 'Change_CANCELLATIONS_BEFORE_NOON',
 'personal',
 'professional']
    
    
    ,'first_dec_tree_prune':['MOBILE_NUMBER',
 'CANCELLATIONS_BEFORE_NOON',
 'FOLLOWED_RECOMMENDATIONS_PCT',
 'AVG_PREP_VID_TIME',
 'existing_users_final_funnel_lo',
 'existing_users_final_funnel_hi_ALTERNATIVE',
 'personal',
 'professional',
 'early_over_total',
 'late_deliveries_over_product_catViewed',
 'total_logins_over_total_meals_ordered',
 'total_unique_orders',
 'total_orders_without_cancellations',
 'late_deli_over_total_meals']
    
}
  


### Prep For Models

Train and test split is done in this section. 
- Test size: 0.25
- Random Seed: 222


In [24]:
lrFeat = df.loc[:,features['Second_log_reg_select_from_model_sklearn']] ### First_log_reg_select_from_model_sklearn


## For full-variable model:
# lrFeat = df.drop(['NAME',
#  'EMAIL',
#  'EMAIL_TYPE',
#  'EMAIL_DOMAIN',
#  'FIRST_NAME',
#  'FAMILY_NAME',
#  'CROSS_SELL_SUCCESS'
 
# ], axis = 1)


lrTarget = df.loc[:,'CROSS_SELL_SUCCESS']


from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA

st_scaler = StandardScaler()
mn_max_scaler = MinMaxScaler()
pca = PCA(n_components=2)


# lrFeat_st_scaled = st_scaler.fit_transform(lrFeat)
# lrFeat_mn_max_scaled = mn_max_scaler.fit_transform(lrFeat)
# lrFeat_pca_scaled = pca.fit_transform(lrFeat)

X_train, X_test, y_train, y_test = train_test_split(lrFeat,
                                                    lrTarget,
                                                    test_size = 0.25,
                                                    random_state = 222,
                                                   stratify = lrTarget)


# # Features
X_train.columns.to_list()

['MOBILE_NUMBER',
 'existing_users_final_funnel_lo',
 'Change_CANCELLATIONS_BEFORE_NOON',
 'personal',
 'professional']

### Base Model - Logistic Regression
~~~
Train Metrics:
            Recall:     0.851 
            Accuracy:   0.777
            AUC:        0.735 
            
____________________________________________________________________
            
    Test Metrics:
            Recall:     0.819 
            Accuracy:   0.749
            AUC:        0.711 
~~~

In [25]:
lr_model = LogisticRegression(C = 1,random_state=222).fit(X_train,y_train)
lr_fit = lr_model.fit(X_train, y_train)

y_pred_train = lr_fit.predict(X_train)
y_pred_test = lr_fit.predict(X_test)


print(confusion_matrix(y_train,y_pred_train))

print(f""" Train Metrics:
            Recall:     {round(recall_score(y_train,y_pred_train),3)} 
            Accuracy:   {round(accuracy_score(y_train, y_pred_train),3)}
            AUC:        {round(roc_auc_score(y_train, y_pred_train),3)} 
            
____________________________________________________________________
            
    Test Metrics:
            Recall:     {round(recall_score(y_test,y_pred_test),3)} 
            Accuracy:   {round(accuracy_score(y_test, y_pred_test),3)}
            AUC:        {round(roc_auc_score(y_test,y_pred_test),3)} """)

lr_coeffs = pd.DataFrame(
    
    lr_fit.coef_.round(2)[0],
    index = X_train.columns.to_list(),
    columns = ['coeffs']

)



# Most 
# lr_coeffs[np.abs(lr_coeffs.coeffs) > 0 ].sort_values('coeffs') 
# Without pruning
lr_coeffs.loc[np.abs(lr_coeffs.coeffs) < 0.05 ].sort_index()

# lr_coeffs.loc[np.abs(lr_coeffs.coeffs) > 0.05 ].sort_index().index.to_list()


[[343 126]
 [165 825]]
 Train Metrics:
            Recall:     0.833 
            Accuracy:   0.801
            AUC:        0.782 
            
____________________________________________________________________
            
    Test Metrics:
            Recall:     0.801 
            Accuracy:   0.782
            AUC:        0.772 


/home/diego/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/diego/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


,coeffs


### Sklearn Feature Selection

In [26]:
from sklearn.feature_selection import SelectFromModel
lr_model = LogisticRegression(C = 1,random_state=222).fit(X_train,y_train)
model = SelectFromModel(lr_model, prefit=True)
X_new = model.transform(X_train)
# LinearSVC(penalty="l1")
X_train.columns[model.get_support()].to_list()

/home/diego/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


['existing_users_final_funnel_lo']

### Model Selection 
Models Tested: Ridge, LogisticRegression, SGDClassifier, KNeighborsClassifier, AdaBoostClassifier, BaggingClassifier, GradientBoostingClassifier, RandomForestClassifier

Winner: Logistic Regression due to its interpretability and boosting/bagging methods' scores didn't differ that much. 




In [38]:
from sklearn.linear_model import RidgeClassifier, LogisticRegression, SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier, GradientBoostingClassifier,RandomForestClassifier
from sklearn.model_selection import KFold, cross_val_score,GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.metrics import recall_score, confusion_matrix,accuracy_score, auc, roc_auc_score


seed = 222
num_folds = 3
scoring = make_scorer(roc_auc_score, needs_threshold = False)
models = []




models.append(('RidgeClassifier', RidgeClassifier()))
models.append(('LogisticRegression', LogisticRegression(solver = 'lbfgs', max_iter = 5000)))
models.append(('SGDClassifier', SGDClassifier()))
models.append(('KNeighborsClassifier',KNeighborsClassifier(n_neighbors = 5)))
# models.append(('ARDRegression', ARDRegression()))
# models.append(('AdaBoostRegressor', AdaBoostRegressor()))
models.append(('AdaBoostClassifier', AdaBoostClassifier(random_state = 222)))
models.append(('BaggingClassifier', BaggingClassifier(random_state = 222)))
models.append(('GradientBoostingClassifier', GradientBoostingClassifier(random_state = 222)))
models.append(('RandomForestClassifier', RandomForestClassifier(n_estimators = 100, 
                                                                random_state = 222)))
results_train = []
results_test = []
names = []
for name, model in models:
    kfold = KFold(n_splits=num_folds, random_state=seed)
    cv_results_train = cross_val_score(model, X_train, y_train, cv=kfold, n_jobs=-1, scoring=scoring)
    cv_results_test = cross_val_score(model, X_test, y_test, cv=kfold, n_jobs=-1,  scoring=scoring)
    results_train.append(cv_results_train.mean())
    results_test.append(cv_results_test.mean())
    names.append(name)
#     msg = "%s %f %f " % (name, cv_results.mean(), cv_results.std())
#     print(f'Training {name}')


models_log = pd.DataFrame({'model_name':names,
                             'train_results':results_train,
                             'test_results':results_test})

models_log


,model_name,train_results,test_results
0,RidgeClassifier,0.782258,0.718031
1,LogisticRegression,0.782258,0.728805
2,SGDClassifier,0.726549,0.702355
3,KNeighborsClassifier,0.746026,0.708179
4,AdaBoostClassifier,0.782258,0.725881
5,BaggingClassifier,0.779553,0.749978
6,GradientBoostingClassifier,0.785749,0.710418
7,RandomForestClassifier,0.779627,0.719921


### Winner Model: Logistic Regression

The Gradient Boosting Parameters were found using GridsearchCV. The hyper-parameters are defined below. 
<br> 

#### GridSearch Logistic Regression

In [28]:
from sklearn.metrics import roc_auc_score, roc_curve, confusion_matrix
from sklearn.metrics import make_scorer              # customizable scorer
from sklearn.model_selection import GridSearchCV

## Params Tested:
params = {
    "penalty": ['l1','l2'],
    'solver':['saga'],
    'C':np.arange(0.01,11,0.1),
    'warm_start':[True,False]
    }


lr_tuned = LogisticRegression(random_state=222)


lr_model = GridSearchCV(lr_tuned,
             param_grid=params,
             n_jobs = -1, 
             scoring= make_scorer(roc_auc_score,needs_threshold=False),
             cv = 3)


lr_model.fit(X_train,y_train)


print(f""" \n Train Metrics:

            CV AUC Score: {round(lr_model.best_score_,3)}
            model Parameters: {lr_model.best_params_},
            best Model: {lr_model.best_estimator_}
            
            """)

 
 Train Metrics:

            CV AUC Score: 0.782
            model Parameters: {'C': 0.21000000000000002, 'penalty': 'l1', 'solver': 'saga', 'warm_start': True},
            best Model: LogisticRegression(C=0.21000000000000002, class_weight=None, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='warn', n_jobs=None, penalty='l1',
                   random_state=222, solver='saga', tol=0.0001, verbose=0,
                   warm_start=True)
            
            


#### Logistic Regressions with final parameters

In [58]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import recall_score, confusion_matrix,accuracy_score, auc, roc_auc_score



lr_model = LogisticRegression(C=0.21, class_weight=None, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='warn', n_jobs=None, penalty='l1',
                   random_state=222, solver='saga', tol=0.0001, verbose=0,
                   warm_start=True)

lr_fit = lr_model.fit(X_train, y_train)

y_pred_train = lr_fit.predict(X_train)
y_pred_test = lr_fit.predict(X_test)


print(confusion_matrix(y_train,y_pred_train))

print(f""" Train Metrics:
            Recall:     {round(recall_score(y_train,y_pred_train),3)} 
            Accuracy:   {round(accuracy_score(y_train, y_pred_train),3)}
            AUC:        {round(roc_auc_score(y_train, y_pred_train),3)} 
            
____________________________________________________________________
            
    Test Metrics:
            Recall:     {round(recall_score(y_test,y_pred_test),3)} 
            Accuracy:   {round(accuracy_score(y_test, y_pred_test),3)}
            AUC:        {round(roc_auc_score(y_test,y_pred_test),3)} """)

lr_coeffs = pd.DataFrame(
    
    lr_fit.coef_.round(2)[0],
    index = X_train.columns.to_list(),
    columns = ['coeffs']

)



# Most 
# lr_coeffs[np.abs(lr_coeffs.coeffs) > 0 ].sort_values('coeffs') 
# Without pruning
# lr_coeffs.loc[np.abs(lr_coeffs.coeffs) >= 0 ].sort_index()
print('coeffs:\n',lr_coeffs)

odds = np.exp(lr_coeffs)
# Probabilities of Coeffs (need to reverse existing users final because is negative)
print('probs:\n',(odds / (1 + odds)))

[[343 126]
 [165 825]]
 Train Metrics:
            Recall:     0.833 
            Accuracy:   0.801
            AUC:        0.782 
            
____________________________________________________________________
            
    Test Metrics:
            Recall:     0.801 
            Accuracy:   0.782
            AUC:        0.772 
coeffs:
                                   coeffs
MOBILE_NUMBER                       0.54
existing_users_final_funnel_lo     -4.87
Change_CANCELLATIONS_BEFORE_NOON    0.68
personal                            0.85
professional                        1.46
probs:
                                     coeffs
MOBILE_NUMBER                     0.631812
existing_users_final_funnel_lo    0.007615
Change_CANCELLATIONS_BEFORE_NOON  0.663739
personal                          0.700567
professional                      0.811533


In [75]:
df.describe().iloc[:,5:16].round(2)

,PRODUCT_CATEGORIES_VIEWED,AVG_TIME_PER_SITE_VISIT,MOBILE_NUMBER,CANCELLATIONS_BEFORE_NOON,CANCELLATIONS_AFTER_NOON,TASTES_AND_PREFERENCES,PC_LOGINS,MOBILE_LOGINS,WEEKLY_PLAN,EARLY_DELIVERIES,LATE_DELIVERIES
count,1946.00,1946.00,1946.00,1946.00,1946.00,1946.00,1946.00,1946.00,1946.00,1946.00,1946.00
mean,5.38,99.60,0.88,1.40,0.17,0.71,5.52,1.48,11.33,1.49,2.97
std,3.04,62.34,0.33,1.55,0.43,0.45,0.58,0.53,13.57,2.32,2.74
min,1.00,10.33,0.00,0.00,0.00,0.00,4.00,0.00,0.00,0.00,0.00
25%,3.00,72.00,1.00,0.00,0.00,0.00,5.00,1.00,1.00,0.00,1.00
50%,5.00,94.16,1.00,1.00,0.00,1.00,6.00,1.00,7.00,0.00,2.00
75%,8.00,117.29,1.00,2.00,0.00,1.00,6.00,2.00,13.00,3.00,4.00
max,10.00,1645.60,1.00,13.00,3.00,1.00,7.00,3.00,52.00,9.00,19.00
